# Пайплайн: запросы => товары => фото товаров => предсказание релевантности => nDCG 

## Libraries installed

In [1]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "sglang[all]"


  Using cached anthropic-0.40.0-py3-none-any.whl.metadata (23 kB)
  Using cached tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached vllm-0.6.4.post1-cp38-abi3-manylinux1_x86_64.whl.metadata (10 kB)
  Using cached cuda_python-12.6.2.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
INFO: pip is looking at multiple versions of sglang[srt] to determine which version is compatible with other requirements. This could take a while.
  Using cached sglang-0.3.6.post3-py3-none-any.whl.metadata (22 kB)
  Using cached sglang-0.3.6.post2-py3-none-any.whl.metadata (22 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Using cached aiohttp-3.1

In [3]:
pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/


Looking in indexes: https://flashinfer.ai/whl/cu121/torch2.4/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 2.2 MB/s eta 0:00:0000:0100:04m
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install 'accelerate>=0.26.0'


  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install elasticsearch elasticsearch6 omegaconf

  Using cached elasticsearch-8.16.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached elasticsearch6-6.8.2-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached elastic_transport-8.15.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
Using cached elasticsearch-8.16.0-py3-none-any.whl (543 kB)
Using cached elasticsearch6-6.8.2-py2.py3-none-any.whl (90 kB)
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
Using cached elastic_transport-8.15.1-py3-none-any.whl (64 kB)


In [6]:
pip install --upgrade elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import json
import pandas as pd
import pycurl
import requests
from tqdm import tqdm
from elasticsearch6 import Elasticsearch
import re
import itertools
import numpy as np
from omegaconf import OmegaConf

# Из файлов в папке Жени беру ID товаров для каждого запроса
в файлах инфа о выданных поиском товарах 

In [8]:
folder_path = "search_responses"
files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
files

[', , , 32 81 hd sber sdx-32h2128 smart tv смотреть телевизор.json',
 '10050fe hhy hyundai бензиновый генератор.json',
 '16 дивай софия.json',
 '2249 китфорт.json',
 '358 shu.json',
 '70mai up.json',
 '95 agm delta.json',
 'airgrid вытяжной зонт.json',
 'dell g2524h монитор.json',
 'fs40 nobo ножка.json',
 'garlyn увлажнитель.json',
 'gorenje mg2001sb.json',
 'maison manons диван угловой.json',
 'ng332014r.json',
 'ra-kv0402s.json',
 'sasakl обруч.json',
 'xgimi z7x.json',
 'zofft фен-стайлер.json',
 'аквариум будда для фигурка.json',
 'гельтечь пилинг сыворотка.json',
 'гринч маска.json',
 'гриньча маска.json',
 'диван кровать оскар.json',
 'диван сеул.json',
 'игоровой руль.json',
 'кровать пора спать.json',
 'металлидти сольфеджио.json',
 'металлидти.json']

In [9]:
data = []

for file in files:
    file_path = os.path.join(folder_path, file)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = json.load(f) 
            
            items = content.get("results", {}).get("items", [])
            query = content.get("request", {}).get("query", "") or file.replace(".json", "")

            for item_id in items:
                data.append({"ID": item_id, "Keyword": query})
    except Exception as e:
        print(f"Ошибка при обработке файла {file}: {e}")

In [10]:
df = pd.DataFrame(data)

In [11]:
df = df.rename(columns={"": "c"})
df

,ID,Keyword
0,100067318660,", , , 32 81 hd sber sdx-32h2128 smart tv смотр..."
1,100067465006,", , , 32 81 hd sber sdx-32h2128 smart tv смотр..."
2,600009041032,", , , 32 81 hd sber sdx-32h2128 smart tv смотр..."
3,100030990679,", , , 32 81 hd sber sdx-32h2128 smart tv смотр..."
4,100030532483,", , , 32 81 hd sber sdx-32h2128 smart tv смотр..."
...,...,...
129,100056593472,металлидти
130,100057475462,металлидти
131,100042450266,металлидти
132,100043127496,металлидти


# Из Elatic search беру фото и название товара

In [16]:
def get_item_by_id(elastic_hosts: str, elastic_index: str, item_id: str) -> dict:
    es = Elasticsearch(hosts=elastic_hosts)
    data_query = {
        "query": {
            "ids": {
                "values": [item_id]
            }
        }
    }
    response = es.search(index=elastic_index, body=data_query)
    if response['hits']['total'] > 0:
        return response['hits']['hits'][0]['_source']
    else:
        return {"error": "Item not found"}


def get_product_name(item_id):
    item_data = get_item_by_id(elastic_hosts, elastic_index, item_id)
    return item_data['long_web_name']

def get_product_photo(item_id):
    item_data = get_item_by_id(elastic_hosts, elastic_index, item_id)
    link = 'https://main-cdn.sbermegamarket.ru/big2' + item_data['images'][0]['image_link']
    return link

In [17]:
elastic_hosts = 'http://elastic-lb-01.prod.lan:19200/'
elastic_index = 'item_index_v7'

In [18]:
df['product_name'] = [get_product_name(id) for id in tqdm(df['ID'].tolist(), desc="Processing queries")]



Processing queries:   0%|          | 0/134 [00:00<?, ?it/s]

Processing queries:   1%|▏         | 2/134 [00:00<00:06, 19.72it/s]

Processing queries:   4%|▎         | 5/134 [00:00<00:05, 21.69it/s]

Processing queries:   6%|▌         | 8/134 [00:00<00:10, 12.44it/s]

Processing queries:   8%|▊         | 11/134 [00:00<00:08, 15.13it/s]

Processing queries:  10%|█         | 14/134 [00:00<00:06, 17.62it/s]

Processing queries:  13%|█▎        | 17/134 [00:00<00:05, 20.01it/s]

Processing queries:  15%|█▍        | 20/134 [00:01<00:08, 13.90it/s]

Processing queries:  17%|█▋        | 23/134 [00:01<00:06, 16.14it/s]

Processing queries:  19%|█▉        | 26/134 [00:01<00:05, 18.25it/s]

Processing queries:  22%|██▏       | 29/134 [00:01<00:05, 19.64it/s]

Processing queries:  24%|██▍       | 32/134 [00:02<00:07, 13.24it/s]

Processing queries:  26%|██▌       | 35/134 [00:02<00:06, 15.49it/s]

Processing queries:  28%|██▊       | 38/134 [00:02<00:05, 16.41it/s]

Processing queries:  31%|███  

In [19]:
df['product_photo_link'] = [get_product_photo(id) for id in tqdm(df['ID'].tolist(), desc="Processing queries")]



Processing queries:   0%|          | 0/134 [00:00<?, ?it/s]

Processing queries:   2%|▏         | 3/134 [00:00<00:07, 17.20it/s]

Processing queries:   4%|▍         | 6/134 [00:00<00:06, 19.86it/s]

Processing queries:   7%|▋         | 9/134 [00:00<00:05, 21.89it/s]

Processing queries:   9%|▉         | 12/134 [00:00<00:05, 23.26it/s]

Processing queries:  11%|█         | 15/134 [00:00<00:07, 16.31it/s]

Processing queries:  13%|█▎        | 18/134 [00:00<00:06, 18.18it/s]

Processing queries:  16%|█▌        | 21/134 [00:01<00:05, 20.27it/s]

Processing queries:  18%|█▊        | 24/134 [00:01<00:04, 22.16it/s]

Processing queries:  20%|██        | 27/134 [00:01<00:04, 23.56it/s]

Processing queries:  22%|██▏       | 30/134 [00:01<00:04, 24.44it/s]

Processing queries:  25%|██▍       | 33/134 [00:01<00:03, 25.41it/s]

Processing queries:  27%|██▋       | 36/134 [00:01<00:04, 24.34it/s]

Processing queries:  29%|██▉       | 39/134 [00:01<00:03, 24.36it/s]

Processing queries:  31%|███▏ 

In [20]:
df

,ID,Keyword,product_name,product_photo_link
0,100067318660,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор Sber SDX-32H2128, 32""(81 см), HD RAM...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
1,100067465006,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор Sber SDX-32F2137, 32""(81 см), FHD RA...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
2,600009041032,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор HYUNDAI H-LED32BS5003, 32""(81 см), HD",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
3,100030990679,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор Haier HEC R1 32 S HD, 32""(81 см), HD",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
4,100030532483,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор HISENSE 32A4BG, 32""(81 см), HD",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
...,...,...,...,...
129,100056593472,металлидти,Сольфеджио. 1 класс. Учебное пособие. Мы играе...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
130,100057475462,металлидти,"Металлиди Ж, А.Перцовская «Мы играем, сочиняем...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
131,100042450266,металлидти,Хрестоматия для саксофона-альта. 5-6 годы обуч...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...
132,100043127496,металлидти,"Металлиди Ж., Перцовская А. Учиться музыке лег...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...


# LLM предсказывает релевантность товара запросу (по названию и фото товара)

In [106]:
df_with_predictions = df.copy()
df_with_predictions['prediction'] = None

In [108]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

results = []

for index, row in df.iterrows():
    system = f"""\
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар, его характеристики и его фото поисковому запросу.

Классы запросов:
* relevant - товары, которые можно купить по указанному запросу.
* irrelevant - товары, которые было бы странно увидеть по указанному запросу.

Определи релевантность товара запросу. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.

Товар:
{row['product_name']}
"""

    system_query = f"""\
Подумай, проанализируй товар (только ничего не пиши). Выдели самые его основные и ключевые характеристики (ничего не пиши, думай).
Так же проанализируй изображение товара (только ничего не пиши).

Запрос:
{row['Keyword']}
"""

    messages = [
        {"role": "user", "content": [{"type": "text", "text": system}]}
    ]

    # ссылка на изображение
    if row['product_photo_link']:
        messages[0]["content"].append(
            {"type": "image_url", "image_url": {"url": row['product_photo_link']}}
        )

    messages[0]["content"].append({"type": "text", "text": system_query})

    try:
        response = client.chat.completions.create(
            model="Qwen2-VL-72B-Instruct-AWQ",
            messages=messages,
        )

        prediction = response.choices[0].message.content.strip().lower()
        df_with_predictions.at[index, 'prediction'] = prediction
    except Exception as e:
        print(f"Error processing item {i}: {e}")




In [109]:
df_with_predictions

,ID,Keyword,product_name,product_photo_link,prediction
0,100067318660,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор Sber SDX-32H2128, 32""(81 см), HD RAM...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant
1,100067465006,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор Sber SDX-32F2137, 32""(81 см), HD RAM...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant
2,600009041032,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор HYUNDAI H-LED32BS5003, 32""(81 см), HD",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant
3,100030990679,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор Haier HEC R1 32 S HD, 32""(81 см), HD",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant
4,100030532483,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...","Телевизор HISENSE 32A4BG, 32""(81 см), HD",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant
...,...,...,...,...,...
129,100056593472,металлидти,Сольфеджио. 1 класс. Учебное пособие. Мы играе...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
130,100057475462,металлидти,"Металлиди Ж, А.Перцовская «Мы играем, сочиняем...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant
131,100042450266,металлидти,Хрестоматия для саксофона-альта. 5-6 годы обуч...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
132,100043127496,металлидти,"Металлиди Ж., Перцовская А. Учиться музыке лег...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,relevant


### Смотрим нерелевантные товары (вроде все норм определилось)

In [110]:
df_with_predictions[df_with_predictions['prediction']=='irrelevant']

,ID,Keyword,product_name,product_photo_link,prediction
12,600004241444,16 дивай софия,Диван для собак FUNTIKSTORE Джоконда 48х61х38 см,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
23,600008565935,95 agm delta,"Аккумулятор автомобильный, Delta, Delta СТ",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
25,600020100270,95 agm delta,Аккумулятор DELTA BATTERY 12201 18А (-/+) обр/...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
26,600020183274,95 agm delta,Аккумулятор DELTA BATTERY 1220 20А (-/+) обр/п...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
27,600020100314,95 agm delta,Аккумулятор DELTA BATTERY 1210.1 10А (+/-) п/п...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
125,100041406431,металлидти,"Ж., Перцовская А. «Учиться музыке легко». 2 кл...",https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
127,100056593473,металлидти,Сольфеджио. 2 класс. Учебное пособие. Мы играе...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
128,600008791415,металлидти,Комплект ученика Учиться музыке легко. 1 класс,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
129,100056593472,металлидти,Сольфеджио. 1 класс. Учебное пособие. Мы играе...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant
131,100042450266,металлидти,Хрестоматия для саксофона-альта. 5-6 годы обуч...,https://main-cdn.sbermegamarket.ru/big2/hlr-sy...,irrelevant


# nDCG метрика ранжирования

In [111]:
from sklearn.metrics import ndcg_score

df_with_predictions2 = df_with_predictions.copy()
df_with_predictions2["rel_score"] = df_with_predictions2["prediction"].apply(lambda x: 1 if x == "relevant" else 0)

In [112]:
ndcg_results = []

for keyword, group in df_with_predictions2.groupby("Keyword"):
    if len(group) > 1: # смотрим только запросы, у которых выдалось больше 1 товара
        print(f'\n{keyword}')
        pred_relevance = group["rel_score"].values.reshape(1, -1)  # предсказанные метки
        print(f'{pred_relevance} (предсказанные)')
        ideal_relevance =  group.sort_values("rel_score", ascending=False)["rel_score"].values.reshape(1, -1) # идеальные метки
        print(f'{ideal_relevance} (идеальные)')
        ndcg = ndcg_score(pred_relevance, ideal_relevance, k=len(group)) 
        ndcg_results.append({
            "Keyword": keyword,
            "total_items": len(group),
            "relevant_items": group["rel_score"].sum(),
            "irrelevant_items": len(group) - group["rel_score"].sum(),
            "nDCG": ndcg
        })


, , , 32 81 hd sber sdx-32h2128 smart tv смотреть телевизор
[[1 1 1 1 1 1 1 1 1 1]] (предсказанные)
[[1 1 1 1 1 1 1 1 1 1]] (идеальные)

10050fe hhy hyundai бензиновый генератор
[[1 1]] (предсказанные)
[[1 1]] (идеальные)

70mai up
[[1 1 1 1 1 1 1]] (предсказанные)
[[1 1 1 1 1 1 1]] (идеальные)

95 agm delta
[[1 0 1 0 0 0 1 1 1 1]] (предсказанные)
[[1 1 1 1 1 1 0 0 0 0]] (идеальные)

airgrid вытяжной зонт
[[1 1 1]] (предсказанные)
[[1 1 1]] (идеальные)

fs40 nobo ножка
[[1 1]] (предсказанные)
[[1 1]] (идеальные)

maison manons диван угловой
[[1 1 1 1 1 1 1 1 1 1]] (предсказанные)
[[1 1 1 1 1 1 1 1 1 1]] (идеальные)

ra-kv0402s
[[1 1 1 1]] (предсказанные)
[[1 1 1 1]] (идеальные)

sasakl обруч
[[1 1 1 1 1 1 1 1 1 1]] (предсказанные)
[[1 1 1 1 1 1 1 1 1 1]] (идеальные)

xgimi z7x
[[1 1]] (предсказанные)
[[1 1]] (идеальные)

аквариум будда для фигурка
[[1 1 1 1 1]] (предсказанные)
[[1 1 1 1 1]] (идеальные)

гринч маска
[[1 1 1 1 1]] (предсказанные)
[[1 1 1 1 1]] (идеальные)

гриньча ма

In [113]:
ndcg_df = pd.DataFrame(ndcg_results)
ndcg_df

,Keyword,total_items,relevant_items,irrelevant_items,nDCG
0,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...",10,10,0,1.000000
1,10050fe hhy hyundai бензиновый генератор,2,2,0,1.000000
2,70mai up,7,7,0,1.000000
3,95 agm delta,10,6,4,0.708225
4,airgrid вытяжной зонт,3,3,0,1.000000
5,fs40 nobo ножка,2,2,0,1.000000
6,maison manons диван угловой,10,10,0,1.000000
7,ra-kv0402s,4,4,0,1.000000
8,sasakl обруч,10,10,0,1.000000
9,xgimi z7x,2,2,0,1.000000
